In [ ]:
pip install ultralytics opencv-python numpy


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 18.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 4.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 98.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 85.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 37.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 2.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 5.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 16.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 7.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 6.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 60.9 MB/s eta 0:00:00
  Attempting uninstall: nvidia-nvjitlink-cu12
    Found existing installation: nvidia-nvjitlink-cu12 12.5.82
    Uninstalling 

In [ ]:
from google.colab import files

uploaded = files.upload()


Saving 15sec_input_720p.mp4 to 15sec_input_720p (1).mp4
Saving best.pt to best.pt


In [ ]:
import os
os.listdir("/content")

['.config',
 'best.pt',
 '15sec_input_720p (1).mp4',
 '15sec_input_720p.mp4',
 'sample_data']

In [ ]:
from ultralytics import YOLO
import cv2
import numpy as np
from google.colab.patches import cv2_imshow


In [ ]:
# Load YOLOv11 model (uploaded file)
model = YOLO("/content/best.pt")

# Load the video
cap = cv2.VideoCapture("/content/15sec_input_720p.mp4")


In [ ]:
class Player:
    def __init__(self, bbox, player_id, color):
        self.bbox = bbox  # [x1, y1, x2, y2]
        self.id = player_id
        self.color = color
        self.last_seen = 0  # Frame number

def iou(box1, box2):
    # Intersection over Union
    x1 = max(box1[0], box2[0])
    y1 = max(box1[1], box2[1])
    x2 = min(box1[2], box2[2])
    y2 = min(box1[3], box2[3])

    inter_area = max(0, x2 - x1) * max(0, y2 - y1)
    area1 = (box1[2]-box1[0]) * (box1[3]-box1[1])
    area2 = (box2[2]-box2[0]) * (box2[3]-box2[1])

    union_area = area1 + area2 - inter_area
    return inter_area / union_area if union_area > 0 else 0


In [10]:
players = []           # List to store tracked players
player_count = 0       # ID counter
frame_num = 0          # Current frame index

while cap.isOpened():
    ret, frame = cap.read()
    if not ret:
        break

    results = model(frame)[0]  # Run YOLO
    current_boxes = []

    # Extract detected players
    for box in results.boxes:
        x1, y1, x2, y2 = map(int, box.xyxy[0])
        cls = int(box.cls[0])  # class index
        conf = float(box.conf[0])
        if cls == 0 and conf > 0.5:  # class 0 = player
            current_boxes.append([x1, y1, x2, y2])

    matched_ids = set()

    # Try to match current detections with existing tracked players
    for box in current_boxes:
        best_iou = 0
        best_player = None

        for player in players:
            iou_score = iou(player.bbox, box)
            if iou_score > best_iou:
                best_iou = iou_score
                best_player = player

        if best_iou > 0.3:
            # Match found — update player
            best_player.bbox = box
            best_player.last_seen = frame_num
            matched_ids.add(best_player.id)
        else:
            # New player — create new ID
            player_count += 1
            color = tuple(np.random.randint(0, 255, 3).tolist())
            new_player = Player(box, player_count, color)
            new_player.last_seen = frame_num
            players.append(new_player)

    # Remove players not seen for 15 frames
    players = [p for p in players if frame_num - p.last_seen <= 15]

    # Draw all current tracked players
    for player in players:
        x1, y1, x2, y2 = player.bbox
        cv2.rectangle(frame, (x1, y1), (x2, y2), player.color, 2)
        cv2.putText(frame, f"ID: {player.id}", (x1, y1 - 10),
                    cv2.FONT_HERSHEY_SIMPLEX, 0.6, player.color, 2)

    # Resize for display
    resized_frame = cv2.resize(frame, (640, 360))
    cv2_imshow(resized_frame)

    frame_num += 1

cap.release()
cv2.destroyAllWindows()


Output hidden; open in https://colab.research.google.com to view.

In [12]:
# OpenCV setup
cap = cv2.VideoCapture("/content/15sec_input_720p.mp4")

# Get original video dimensions and FPS
width  = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
height = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))
fps    = cap.get(cv2.CAP_PROP_FPS)

# Define video writer
fourcc = cv2.VideoWriter_fourcc(*'XVID')  # or 'mp4v' for .mp4
out = cv2.VideoWriter('output.avi', fourcc, fps, (width, height))

players = []
player_count = 0
frame_num = 0

while cap.isOpened():
    ret, frame = cap.read()
    if not ret:
        break

    results = model(frame)[0]
    current_boxes = []

    for box in results.boxes:
        x1, y1, x2, y2 = map(int, box.xyxy[0])
        cls = int(box.cls[0])
        conf = float(box.conf[0])
        if cls == 0 and conf > 0.5:
            current_boxes.append([x1, y1, x2, y2])

    matched_ids = set()

    for box in current_boxes:
        best_iou = 0
        best_player = None

        for player in players:
            iou_score = iou(player.bbox, box)
            if iou_score > best_iou:
                best_iou = iou_score
                best_player = player

        if best_iou > 0.3:
            best_player.bbox = box
            best_player.last_seen = frame_num
            matched_ids.add(best_player.id)
        else:
            player_count += 1
            color = tuple(np.random.randint(0, 255, 3).tolist())
            new_player = Player(box, player_count, color)
            new_player.last_seen = frame_num
            players.append(new_player)

    players = [p for p in players if frame_num - p.last_seen <= 15]

    for player in players:
        x1, y1, x2, y2 = player.bbox
        cv2.rectangle(frame, (x1, y1), (x2, y2), player.color, 2)
        cv2.putText(frame, f"ID: {player.id}", (x1, y1 - 10),
                    cv2.FONT_HERSHEY_SIMPLEX, 0.6, player.color, 2)

    out.write(frame)  # 🔴 Save the frame to output file

    frame_num += 1

cap.release()
out.release()



0: 384x640 1 ball, 16 players, 2 referees, 2785.3ms
Speed: 4.4ms preprocess, 2785.3ms inference, 1.8ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 18 players, 2 referees, 2646.8ms
Speed: 4.1ms preprocess, 2646.8ms inference, 1.1ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 1 ball, 16 players, 2 referees, 2625.9ms
Speed: 3.0ms preprocess, 2625.9ms inference, 1.1ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 1 ball, 14 players, 2 referees, 3007.4ms
Speed: 2.9ms preprocess, 3007.4ms inference, 1.4ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 1 ball, 14 players, 2 referees, 3417.1ms
Speed: 3.3ms preprocess, 3417.1ms inference, 1.4ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 1 ball, 16 players, 2 referees, 2645.2ms
Speed: 3.0ms preprocess, 2645.2ms inference, 1.1ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 15 players, 2 referees, 2660.5ms
Speed: 2.9ms preprocess, 2660.5ms inference, 1.1ms 

In [14]:
from google.colab import files
files.download("/content/output.avi")

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>